# YouTube 及多平台下載方案 - 實際測試

測試三種下載方案在不同平台的實際表現：

1. **pytubefix** - YouTube 專用
2. **yt-dlp** - 多平台通用
3. **Playwright** - 瀏覽器模擬（備選）

## 測試 URL 列表

- YouTube MP3: https://www.youtube.com/watch?v=0rp3pP2Xwhs
- YouTube MP4: https://www.youtube.com/shorts/vGmdigoUJ6c
- Facebook Video: https://www.facebook.com/share/r/1Bx14m8c7b/
- Instagram Video: https://www.instagram.com/reel/DRtrroMkeDU/
- X (Twitter) Video: https://x.com/Appraiser008/status/1995090648689717458
- **Threads 公開影片**: https://www.threads.com/@zuck/post/DHV7vTivqWD
- **Threads 輪播貼文**: https://www.threads.com/@zuck/post/DJDhoQfxb43

## Threads 下載研究摘要

根據 yt-dlp GitHub issues 研究結果：

### 目前狀態
- **yt-dlp 官方尚未合併 Threads 支援** (截至 2024/12)
- 有兩個開放的 PR 正在進行中:
  - [PR #9852](https://github.com/yt-dlp/yt-dlp/pull/9852) - 基礎 Threads extractor
  - [PR #13512](https://github.com/yt-dlp/yt-dlp/pull/13512) - 更完整的版本，支援登入

### 登入/認證問題
Threads 對部分內容需要登入驗證：
1. **公開貼文** - 可直接下載，無需登入
2. **私人貼文** - 需要透過 cookies 認證
3. **認證方式**: 使用 `--cookies` 或 `--cookies-from-browser` 參數

### 暫時解決方案
1. **使用開發版 yt-dlp** - 從 PR branch 安裝
2. **手動使用 Threads PR 的 extractor** - 複製 threads.py 到本地
3. **使用 Instagram cookies** - Threads 與 Instagram 共用認證 (同屬 Meta)

In [ ]:
from pathlib import Path
import time

# 設置輸出目錄
OUTPUT_DIR = Path('./downloads')
OUTPUT_DIR.mkdir(exist_ok=True)

# 定義測試 URL
TEST_URLS = {
    "youtube_mp3": "https://www.youtube.com/watch?v=0rp3pP2Xwhs",
    "youtube_mp4": "https://www.youtube.com/shorts/vGmdigoUJ6c",
    "facebook": "https://www.facebook.com/share/r/1Bx14m8c7b/",
    "instagram": "https://www.instagram.com/reel/DRtrroMkeDU/?utm_source=ig_web_copy_link&igsh=NTc4MTIwNjQ2YQ==",
    "twitter": "https://x.com/Appraiser008/status/1995090648689717458?s=20",
    # Threads 測試 URL
    "threads_public": "https://www.threads.com/@zuck/post/DHV7vTivqWD",  # 公開影片貼文
    "threads_private": "https://www.threads.com/@thuy.11.2/post/DRysaLcgNsN?xmt=AQF08iq6Xg6BuyiMMn-9T5oiGmQ6346nnycIlzAzr5C5SQ",  # 多圖片/影片輪播
}

print(f"輸出目錄: {OUTPUT_DIR.absolute()}")
print("\n測試 URL:")
for platform, url in TEST_URLS.items():
    print(f"  {platform:20}: {url}")

# 測試結果記錄
test_results = {}

## pytubefix 測試

pytubefix 是 pytube 的維護版本，測試 YouTube 下載性能

In [ ]:
# 安裝 pytubefix
!pip install -q pytubefix

In [ ]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import subprocess
import re

def test_pytubefix_youtube_mp3():
    """測試 pytubefix YouTube MP3 下載"""
    print("\n" + "="*80)
    print("🧪 測試 1: pytubefix - YouTube MP3 下載")
    print("="*80)
    
    url = TEST_URLS['youtube_mp3']
    start_time = time.time()
    
    try:
        print(f"URL: {url}\n")
        
        # 建立 YouTube 物件
        yt = YouTube(url, on_progress_callback=on_progress)
        
        print(f"標題: {yt.title}")
        print(f"長度: {yt.length} 秒")
        print(f"作者: {yt.author}")
        
        # 清理檔名
        safe_title = re.sub(r'[<>:"/\\|?*]', '_', yt.title)
        
        # 下載音訊
        audio_stream = yt.streams.get_audio_only()
        print(f"音訊串流: {audio_stream}")
        
        output_file = audio_stream.download(
            output_path=str(OUTPUT_DIR),
            filename=f"{safe_title}.m4a"
        )
        
        print(f"✅ 下載完成 (m4a): {output_file}")
        
        # 轉換為 MP3
        try:
            mp3_file = str(OUTPUT_DIR / f"{safe_title}.mp3")
            subprocess.run(
                ['ffmpeg', '-i', output_file, '-q:a', '0', '-map', 'a', mp3_file],
                capture_output=True,
                check=True
            )
            Path(output_file).unlink()
            print(f"✅ 轉換成功 (MP3): {mp3_file}")
            
            elapsed_time = time.time() - start_time
            file_size = Path(mp3_file).stat().st_size / (1024 * 1024)
            
            test_results['pytubefix_youtube_mp3'] = {
                'status': '✅ 成功',
                'time': f"{elapsed_time:.1f}s",
                'size': f"{file_size:.2f}MB",
                'file': mp3_file
            }
            
            return True
            
        except (FileNotFoundError, subprocess.CalledProcessError) as e:
            print(f"⚠️ ffmpeg 不可用: {e}")
            test_results['pytubefix_youtube_mp3'] = {
                'status': '⚠️ 部分成功 (m4a)',
                'file': output_file
            }
            return False
            
    except Exception as e:
        print(f"❌ 下載失敗: {e}")
        test_results['pytubefix_youtube_mp3'] = {
            'status': f'❌ 失敗: {str(e)[:50]}'
        }
        import traceback
        traceback.print_exc()
        return False

def test_pytubefix_youtube_mp4():
    """測試 pytubefix YouTube MP4 下載（最高品質）"""
    print("\n" + "="*80)
    print("🧪 測試: pytubefix - YouTube MP4 下載")
    print("="*80)
    
    url = TEST_URLS['youtube_mp4']
    start_time = time.time()
    
    try:
        print(f"URL: {url}\n")
        
        # 建立 YouTube 物件
        yt = YouTube(url, on_progress_callback=on_progress)
        
        print(f"標題: {yt.title}")
        print(f"長度: {yt.length} 秒")
        
        # 清理檔名
        safe_title = re.sub(r'[<>:"/\\|?*]', '_', yt.title)
        
        # 取得最高品質的視訊（包含音訊）
        # 先找最高品質的 progressive（自帶音訊）流
        stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
        
        if stream is None:
            # 如果沒有 progressive，則分別下載視訊和音訊後合併
            print("⚠️ 沒有 progressive 流，嘗試分別下載視訊和音訊...")
            
            # 下載最高品質視訊
            video_stream = yt.streams.filter(mime_type='video/mp4').order_by('resolution').desc().first()
            audio_stream = yt.streams.filter(mime_type='audio/mp4').order_by('abr').desc().first()
            
            if video_stream is None or audio_stream is None:
                print("❌ 找不到適合的視訊或音訊流")
                test_results['pytubefix_youtube_mp4'] = {'status': '❌ 失敗: 找不到視訊或音訊流'}
                return False
            
            print(f"視訊: {video_stream}")
            print(f"音訊: {audio_stream}")
            
            # 下載視訊
            video_file = video_stream.download(
                output_path=str(OUTPUT_DIR),
                filename=f"{safe_title}_video.mp4"
            )
            
            # 下載音訊
            audio_file = audio_stream.download(
                output_path=str(OUTPUT_DIR),
                filename=f"{safe_title}_audio.m4a"
            )
            
            # 合併視訊和音訊
            final_file = str(OUTPUT_DIR / f"{safe_title}.mp4")
            cmd = [
                'ffmpeg',
                '-i', video_file,
                '-i', audio_file,
                '-c:v', 'copy',
                '-c:a', 'aac',
                '-strict', 'experimental',
                final_file
            ]
            
            subprocess.run(cmd, capture_output=True, check=True)
            
            # 刪除臨時文件
            Path(video_file).unlink()
            Path(audio_file).unlink()
            
            print(f"✅ 合併成功: {final_file}")
            
        else:
            print(f"✅ 找到最高品質流: {stream}")
            
            final_file = stream.download(
                output_path=str(OUTPUT_DIR),
                filename=f"{safe_title}.mp4"
            )
            
            print(f"✅ 下載完成: {final_file}")
        
        elapsed_time = time.time() - start_time
        file_size = Path(final_file).stat().st_size / (1024 * 1024)
        
        test_results['pytubefix_youtube_mp4'] = {
            'status': '✅ 成功',
            'time': f"{elapsed_time:.1f}s",
            'size': f"{file_size:.2f}MB",
            'file': final_file
        }
        
        return True
        
    except Exception as e:
        print(f"❌ 下載失敗: {e}")
        test_results['pytubefix_youtube_mp4'] = {
            'status': f'❌ 失敗: {str(e)[:50]}'
        }
        import traceback
        traceback.print_exc()
        return False

# 執行測試
test_pytubefix_youtube_mp3()

## yt-dlp 多平台測試

測試 yt-dlp 在各平台的下載能力

In [ ]:
from yt_dlp import YoutubeDL
from pathlib import Path

def convert_mp4_for_mobile(input_file, output_file=None, target_bitrate='1000k', target_height=720):
    """
    轉碼 MP4 以適配手機分享
    
    參數:
        input_file: 輸入 MP4 文件
        output_file: 輸出文件 (default: input_file + _mobile.mp4)
        target_bitrate: 目標比特率 (default: 1000k)
        target_height: 目標高度 (default: 720p)
    """
    input_path = Path(input_file)
    if output_file is None:
        output_file = str(input_path.parent / f"{input_path.stem}_mobile.mp4")
    
    print("\n📹 轉碼 MP4 以適配手機分享...")
    print(f"   輸入: {input_path.name}")
    print(f"   輸出: {Path(output_file).name}")
    print(f"   目標: {target_height}p @ {target_bitrate}")
    
    try:
        cmd = [
            'ffmpeg',
            '-i', str(input_file),
            '-vf', f'scale=-2:{target_height}',  # 保持縱橫比，高度為 target_height，寬度必須能被 2 整除
            '-b:v', target_bitrate,
            '-b:a', '128k',
            '-c:v', 'libx264',
            '-c:a', 'aac',
            '-crf', '23',  # 質量 (0-51, 預設 23, 越低越好)
            '-preset', 'medium',  # 編碼速度
            output_file
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        if result.returncode == 0:
            output_size = Path(output_file).stat().st_size / (1024 * 1024)
            print(f"✅ 轉碼成功: {output_size:.2f}MB")
            return output_file
        else:
            print(f"❌ 轉碼失敗: {result.stderr[:200]}")
            return None
            
    except subprocess.TimeoutExpired:
        print("❌ 轉碼超時")
        return None
    except Exception as e:
        print(f"❌ 轉碼錯誤: {e}")
        return None

def test_ytdlp_platform(platform_name, url, enable_mobile_conversion=False):
    """測試 yt-dlp 非 YouTube 平台"""
    print(f"\n🧪 測試: yt-dlp - {platform_name}")
    print("-" * 80)
    
    start_time = time.time()
    
    try:
        print(f"URL: {url}\n")
        
        # 優先選擇有視訊的 MP4 格式
        ydl_opts = {
            'format': 'best[ext=mp4][vcodec!^=none][acodec!^=none]/best[vcodec!^=none][acodec!^=none]/best',
            'outtmpl': str(OUTPUT_DIR / '%(title)s.%(ext)s'),  # 添加副檔名
            'http_headers': {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            },
            'quiet': False,
            'no_warnings': False,
        }
        
        if 'instagram' in platform_name.lower():
            ydl_opts['extractor_args'] = {
                'instagram': {'fetch_all_comments': False}
            }
        
        with YoutubeDL(ydl_opts) as ydl:
            print("開始下載...")
            info = ydl.extract_info(url, download=True)
            
            # 獲取下載的文件
            ext = info.get('ext', 'mp4')
            filename = f"{info.get('title', 'video')}.{ext}"
            filepath = OUTPUT_DIR / filename
            
            elapsed_time = time.time() - start_time
            
            print("\n✅ 下載成功:")
            print(f"   標題: {info.get('title', 'N/A')}")
            print(f"   時間: {elapsed_time:.1f}s")
            print(f"   文件: {filename}")
            print(f"   副檔名: .{ext}")
            
            # MP4 轉碼以適配手機分享
            mobile_file = None
            if enable_mobile_conversion and ext == 'mp4':
                mobile_file = convert_mp4_for_mobile(str(filepath))
            
            test_results[f'ytdlp_{platform_name.replace(" ", "_").lower()}'] = {
                'status': '✅ 成功',
                'title': info.get('title', 'N/A'),
                'time': f"{elapsed_time:.1f}s",
                'file': filename,
                'mobile_file': Path(mobile_file).name if mobile_file else None
            }
            
            return True
            
    except Exception as e:
        elapsed_time = time.time() - start_time
        error_msg = str(e)[:100]
        print(f"❌ 下載失敗: {error_msg}")
        
        test_results[f'ytdlp_{platform_name.replace(" ", "_").lower()}'] = {
            'status': f'❌ 失敗: {error_msg}'
        }
        
        return False

# yt-dlp 只用於非 YouTube 平台
print("\n" + "="*80)
print("🧪 方案 2: yt-dlp (Facebook、Instagram、X/Twitter)")
print("="*80)

## 多平台測試

按順序測試各個平台

In [ ]:
# 首先用 pytubefix 下載 YouTube MP4
print("\n【方案 1: pytubefix - YouTube】")
test_pytubefix_youtube_mp4()

# 然後用 yt-dlp 下載其他平台（啟用手機轉碼）
print("\n【方案 2: yt-dlp - 其他平台】")
print("\n" + "-"*80)
test_ytdlp_platform('Facebook', TEST_URLS['facebook'], enable_mobile_conversion=True)

print("\n" + "-"*80)
test_ytdlp_platform('Instagram', TEST_URLS['instagram'], enable_mobile_conversion=True)

print("\n" + "-"*80)
test_ytdlp_platform('X (Twitter)', TEST_URLS['twitter'], enable_mobile_conversion=True)

## Threads 下載測試

測試 Threads 影片下載的可行性。由於 yt-dlp 官方尚未合併 Threads 支援，我們需要使用開發中的 extractor 或手動方式。

### 方法一：使用開發版 yt-dlp (從 PR branch)

In [ ]:
import subprocess
import json
import re
from pathlib import Path

def install_ytdlp_threads_branch():
    """安裝包含 Threads extractor 的開發版 yt-dlp"""
    print("="*80)
    print("🔧 安裝開發版 yt-dlp (含 Threads 支援)")
    print("="*80)
    
    # 方法1: 從 Enucatl 的 PR branch 安裝 (功能最完整)
    branch_url = "https://github.com/Enucatl/yt-dlp/archive/refs/heads/threads.zip"
    
    try:
        # 先卸載現有版本，再安裝開發版
        print("從 PR branch 安裝開發版 yt-dlp...")
        result = subprocess.run(
            ['pip', 'install', '--force-reinstall', f'git+https://github.com/Enucatl/yt-dlp.git@threads'],
            capture_output=True,
            text=True,
            timeout=120
        )
        
        if result.returncode == 0:
            print("✅ 開發版 yt-dlp 安裝成功！")
            return True
        else:
            print(f"❌ 安裝失敗: {result.stderr}")
            return False
            
    except Exception as e:
        print(f"❌ 安裝發生錯誤: {e}")
        return False

def test_threads_with_ytdlp(url, use_cookies=False, cookies_file=None):
    """使用 yt-dlp 測試 Threads 下載"""
    print(f"\n🧪 測試: yt-dlp Threads 下載")
    print("-" * 80)
    print(f"URL: {url}")
    
    start_time = time.time()
    
    try:
        from yt_dlp import YoutubeDL
        
        ydl_opts = {
            'format': 'best[ext=mp4][vcodec!^=none][acodec!^=none]/best[vcodec!^=none][acodec!^=none]/best',
            'outtmpl': str(OUTPUT_DIR / '%(title)s.%(ext)s'),
            'http_headers': {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            },
            'quiet': False,
            'no_warnings': False,
        }
        
        # 如果需要登入，使用 cookies
        if use_cookies and cookies_file:
            ydl_opts['cookiefile'] = cookies_file
            print(f"使用 cookies 檔案: {cookies_file}")
        
        with YoutubeDL(ydl_opts) as ydl:
            # 首先只提取資訊，不下載
            print("\n📋 提取影片資訊...")
            info = ydl.extract_info(url, download=False)
            
            if info:
                print(f"\n✅ 成功提取資訊:")
                print(f"   標題: {info.get('title', 'N/A')}")
                print(f"   作者: {info.get('uploader', info.get('channel', 'N/A'))}")
                print(f"   描述: {(info.get('description', 'N/A') or 'N/A')[:100]}...")
                print(f"   類型: {'輪播貼文' if info.get('_type') == 'playlist' else '單一媒體'}")
                
                if info.get('_type') == 'playlist':
                    print(f"   項目數: {info.get('playlist_count', len(info.get('entries', [])))}")
                
                # 詢問是否下載
                print("\n📥 開始下載...")
                info = ydl.extract_info(url, download=True)
                
                elapsed_time = time.time() - start_time
                
                test_results['threads_ytdlp'] = {
                    'status': '✅ 成功',
                    'title': info.get('title', 'N/A'),
                    'time': f"{elapsed_time:.1f}s",
                }
                return True
            else:
                raise Exception("無法提取影片資訊")
                
    except Exception as e:
        elapsed_time = time.time() - start_time
        error_msg = str(e)[:100]
        print(f"\n❌ 下載失敗: {error_msg}")
        
        # 檢查是否是認證問題
        if 'login' in error_msg.lower() or 'private' in error_msg.lower():
            print("\n💡 提示: 這可能是私人貼文，需要登入認證")
            print("   使用方式: --cookies cookies.txt 或 --cookies-from-browser chrome")
        
        test_results['threads_ytdlp'] = {
            'status': f'❌ 失敗: {error_msg}',
        }
        
        import traceback
        traceback.print_exc()
        return False

# 執行安裝
install_ytdlp_threads_branch()  # 取消註解以安裝開發版

### 方法二：手動解析 Threads 頁面資料

In [ ]:
def parse_threads_manually(url, debug=False):
    """
    手動解析 Threads 頁面資料 (不依賴 yt-dlp Threads extractor)
    
    原理:
    1. 下載 Threads 頁面 HTML
    2. 從 <script type="application/json" data-sjs> 標籤中提取 JSON 資料
    3. 解析 RelayPrefetchedStreamCache 中的貼文資料
    4. 提取影片 URL
    """
    import re
    import json
    import requests
    from urllib.parse import urlparse
    
    print(f"\n🧪 測試: 手動解析 Threads")
    print("-" * 80)
    print(f"URL: {url}")
    
    start_time = time.time()
    
    try:
        # 從 URL 提取貼文 ID
        match = re.search(r'/(?:post|t)/([^/?#&]+)', url)
        if not match:
            raise Exception("無法從 URL 提取貼文 ID")
        
        post_id = match.group(1)
        print(f"貼文 ID: {post_id}")
        
        # 下載頁面
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
        }
        
        print("下載頁面...")
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        webpage = response.text
        print(f"頁面大小: {len(webpage)} bytes")
        
        # 搜尋包含貼文資料的 JSON script
        json_scripts = re.findall(
            r'<script type="application/json"[^>]*?\sdata-sjs[^>]*?>(.*?)<\s*/script\s*>',
            webpage,
            re.DOTALL | re.IGNORECASE,
        )
        
        print(f"找到 {len(json_scripts)} 個 JSON script 標籤")
        
        # 嘗試多種解析策略
        json_data = None
        main_post = None
        
        for idx, script in enumerate(json_scripts):
            # 策略 1: 檢查是否包含貼文 ID 和關鍵字
            if post_id not in script:
                continue
            
            if debug:
                print(f"\n  [Script {idx}] 包含貼文 ID，長度: {len(script)} bytes")
            
            try:
                parsed = json.loads(script)
                
                # 策略 2: 尋找包含 video_versions 的資料
                def find_video_data(data, depth=0):
                    if depth > 30:
                        return None
                    
                    if isinstance(data, dict):
                        # 檢查是否有 video_versions (影片資料)
                        if 'video_versions' in data and data.get('code') == post_id:
                            return data
                        if 'video_versions' in data and data.get('video_versions'):
                            # 即使沒有 code，也可能是我們要的資料
                            if debug:
                                print(f"    找到 video_versions 在深度 {depth}")
                            return data
                        
                        for key, value in data.items():
                            result = find_video_data(value, depth + 1)
                            if result:
                                return result
                    
                    elif isinstance(data, list):
                        for item in data:
                            result = find_video_data(item, depth + 1)
                            if result:
                                return result
                    
                    return None
                
                result = find_video_data(parsed)
                if result:
                    main_post = result
                    json_data = parsed
                    print(f"  ✅ 在 Script {idx} 中找到影片資料")
                    break
                    
            except json.JSONDecodeError as e:
                if debug:
                    print(f"  [Script {idx}] JSON 解析失敗: {e}")
                continue
        
        # 策略 3: 如果上面都沒找到，嘗試直接從 HTML 中提取影片 URL
        if not main_post:
            print("\n嘗試直接從 HTML 提取影片 URL...")
            
            # 尋找 video_url 或 video_versions
            video_url_match = re.search(
                r'"video_url"\s*:\s*"([^"]+)"',
                webpage
            )
            
            if video_url_match:
                video_url = video_url_match.group(1).replace('\\u0026', '&').replace('\\/', '/')
                print(f"  找到 video_url: {video_url[:80]}...")
                
                # 下載影片
                print(f"\n📥 下載影片...")
                video_response = requests.get(video_url, headers=headers, stream=True)
                
                safe_title = re.sub(r'[<>:"/\\|?*]', '_', post_id)
                output_file = OUTPUT_DIR / f"threads_{safe_title}.mp4"
                
                with open(output_file, 'wb') as f:
                    for chunk in video_response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                file_size = output_file.stat().st_size / (1024 * 1024)
                elapsed_time = time.time() - start_time
                
                print(f"✅ 下載完成: {output_file}")
                print(f"   檔案大小: {file_size:.2f} MB")
                print(f"   花費時間: {elapsed_time:.1f}s")
                
                test_results['threads_manual'] = {
                    'status': '✅ 成功',
                    'time': f"{elapsed_time:.1f}s",
                    'size': f"{file_size:.2f}MB",
                    'file': str(output_file),
                }
                return True
            
            # 嘗試尋找 CDN 影片連結
            cdn_match = re.search(
                r'(https?://[^"]*?scontent[^"]*?\.mp4[^"]*)',
                webpage
            )
            
            if cdn_match:
                video_url = cdn_match.group(1).replace('\\u0026', '&').replace('\\/', '/')
                print(f"  找到 CDN video URL: {video_url[:80]}...")
                
                # 下載影片
                print(f"\n📥 下載影片...")
                video_response = requests.get(video_url, headers=headers, stream=True)
                
                safe_title = re.sub(r'[<>:"/\\|?*]', '_', post_id)
                output_file = OUTPUT_DIR / f"threads_{safe_title}.mp4"
                
                with open(output_file, 'wb') as f:
                    for chunk in video_response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                file_size = output_file.stat().st_size / (1024 * 1024)
                elapsed_time = time.time() - start_time
                
                print(f"✅ 下載完成: {output_file}")
                print(f"   檔案大小: {file_size:.2f} MB")
                print(f"   花費時間: {elapsed_time:.1f}s")
                
                test_results['threads_manual'] = {
                    'status': '✅ 成功',
                    'time': f"{elapsed_time:.1f}s",
                    'size': f"{file_size:.2f}MB",
                    'file': str(output_file),
                }
                return True
            
            raise Exception("無法從頁面提取 JSON 資料或影片 URL，可能需要登入")
        
        print("✅ 成功提取 JSON 資料")
        
        print(f"\n📋 貼文資訊:")
        print(f"   作者: {main_post.get('user', {}).get('username', 'N/A')}")
        caption_text = main_post.get('caption', {})
        if isinstance(caption_text, dict):
            caption_text = caption_text.get('text', 'N/A')
        print(f"   說明: {(caption_text or 'N/A')[:100]}...")
        
        # 提取影片 URL
        video_urls = []
        
        def extract_video_urls(media_data):
            if media_data.get('video_versions'):
                for video in media_data.get('video_versions', []):
                    if video.get('url'):
                        video_urls.append({
                            'url': video.get('url'),
                            'width': video.get('width') or 0,
                            'height': video.get('height') or 0,
                        })
        
        # 檢查是否為輪播
        if main_post.get('carousel_media'):
            print(f"   類型: 輪播貼文 ({len(main_post.get('carousel_media', []))} 項)")
            for media in main_post.get('carousel_media', []):
                extract_video_urls(media)
        else:
            print(f"   類型: 單一媒體")
            extract_video_urls(main_post)
        
        if video_urls:
            print(f"\n📹 找到 {len(video_urls)} 個影片:")
            for i, video in enumerate(video_urls, 1):
                w = video.get('width', 0) or 'N/A'
                h = video.get('height', 0) or 'N/A'
                print(f"   {i}. {w}x{h}")
                print(f"      URL: {video.get('url')[:80]}...")
            
            # 下載第一個影片（或最高品質）
            # 使用安全的比較方式
            def get_video_size(v):
                w = v.get('width') or 0
                h = v.get('height') or 0
                return w * h
            
            best_video = max(video_urls, key=get_video_size) if video_urls else video_urls[0]
            
            print(f"\n📥 下載影片...")
            video_response = requests.get(best_video['url'], headers=headers, stream=True)
            
            safe_title = re.sub(r'[<>:"/\\|?*]', '_', post_id)
            output_file = OUTPUT_DIR / f"threads_{safe_title}.mp4"
            
            with open(output_file, 'wb') as f:
                for chunk in video_response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            file_size = output_file.stat().st_size / (1024 * 1024)
            elapsed_time = time.time() - start_time
            
            print(f"✅ 下載完成: {output_file}")
            print(f"   檔案大小: {file_size:.2f} MB")
            print(f"   花費時間: {elapsed_time:.1f}s")
            
            test_results['threads_manual'] = {
                'status': '✅ 成功',
                'time': f"{elapsed_time:.1f}s",
                'size': f"{file_size:.2f}MB",
                'file': str(output_file),
            }
            return True
        else:
            print("\n⚠️ 此貼文不包含影片")
            test_results['threads_manual'] = {
                'status': '⚠️ 無影片內容',
            }
            return False
            
    except Exception as e:
        elapsed_time = time.time() - start_time
        error_msg = str(e)[:100]
        print(f"\n❌ 解析失敗: {error_msg}")
        
        test_results['threads_manual'] = {
            'status': f'❌ 失敗: {error_msg}',
        }
        
        import traceback
        traceback.print_exc()
        return False

parse_threads_manually(TEST_URLS["threads_public"])

### 方法三： 使用 Instagram Cookies 認證 Threads

Threads 與 Instagram 同屬 Meta 平台，共用認證系統。可使用 Instagram cookies 來存取需要登入的 Threads 內容。

#### 步驟：
1. 在瀏覽器登入 Instagram/Threads
2. 使用瀏覽器擴充套件匯出 cookies (如 "Get cookies.txt LOCALLY")
3. 或使用 yt-dlp 的 `--cookies-from-browser chrome` 參數

In [ ]:
def parse_threads_with_cookies(url, cookies_path=None, debug=False):
    """
    使用 cookies 解析 Threads 頁面資料（支援私人貼文）
    
    參數:
        url: Threads 貼文 URL
        cookies_path: cookies 檔案路徑 (Netscape 格式)
        debug: 是否輸出除錯資訊
    """
    import re
    import json
    import requests
    from http.cookiejar import MozillaCookieJar
    
    print(f"\n🧪 測試: 手動解析 Threads (含 cookies)")
    print("-" * 80)
    print(f"URL: {url}")
    
    start_time = time.time()
    
    try:
        # 從 URL 提取貼文 ID
        match = re.search(r'/(?:post|t)/([^/?#&]+)', url)
        if not match:
            raise Exception("無法從 URL 提取貼文 ID")
        
        post_id = match.group(1)
        print(f"貼文 ID: {post_id}")
        
        # 設定 session
        session = requests.Session()
        
        # 載入 cookies
        if cookies_path and Path(cookies_path).exists():
            print(f"載入 cookies: {cookies_path}")
            try:
                cookie_jar = MozillaCookieJar(str(cookies_path))
                cookie_jar.load(ignore_discard=True, ignore_expires=True)
                session.cookies = cookie_jar
                print(f"   已載入 {len(cookie_jar)} 個 cookies")
            except Exception as e:
                print(f"   ⚠️ 載入 cookies 失敗: {e}")
        
        # 下載頁面
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
        }
        
        print("下載頁面...")
        response = session.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        webpage = response.text
        print(f"頁面大小: {len(webpage)} bytes")
        
        # 檢查是否需要登入
        if 'login' in webpage.lower() and 'Log in to see this post' in webpage:
            print("⚠️ 此貼文需要登入才能查看")
            if not cookies_path:
                raise Exception("需要登入認證，請提供 cookies 檔案")
        
        # 搜尋包含貼文資料的 JSON script
        json_scripts = re.findall(
            r'<script type="application/json"[^>]*?\sdata-sjs[^>]*?>(.*?)<\s*/script\s*>',
            webpage,
            re.DOTALL | re.IGNORECASE,
        )
        
        print(f"找到 {len(json_scripts)} 個 JSON script 標籤")
        
        # 解析策略
        main_post = None
        video_urls = []
        image_urls = []
        
        for idx, script in enumerate(json_scripts):
            if post_id not in script:
                continue
            
            if debug:
                print(f"\n  [Script {idx}] 包含貼文 ID，長度: {len(script)} bytes")
            
            try:
                parsed = json.loads(script)
                
                def find_media_data(data, depth=0):
                    if depth > 30:
                        return None
                    
                    if isinstance(data, dict):
                        # 檢查影片
                        if 'video_versions' in data and data.get('video_versions'):
                            return data
                        # 檢查圖片
                        if 'image_versions2' in data and not data.get('video_versions'):
                            return data
                        
                        for key, value in data.items():
                            result = find_media_data(value, depth + 1)
                            if result:
                                return result
                    
                    elif isinstance(data, list):
                        for item in data:
                            result = find_media_data(item, depth + 1)
                            if result:
                                return result
                    
                    return None
                
                # 尋找所有媒體
                def find_all_media(data, media_list=None, depth=0):
                    if media_list is None:
                        media_list = []
                    if depth > 30:
                        return media_list
                    
                    if isinstance(data, dict):
                        if 'video_versions' in data and data.get('video_versions'):
                            media_list.append({'type': 'video', 'data': data})
                        elif 'image_versions2' in data and data.get('image_versions2'):
                            media_list.append({'type': 'image', 'data': data})
                        
                        for key, value in data.items():
                            find_all_media(value, media_list, depth + 1)
                    
                    elif isinstance(data, list):
                        for item in data:
                            find_all_media(item, media_list, depth + 1)
                    
                    return media_list
                
                media_items = find_all_media(parsed)
                if media_items:
                    print(f"  ✅ 在 Script {idx} 中找到 {len(media_items)} 個媒體")
                    
                    for media in media_items:
                        if media['type'] == 'video':
                            for video in media['data'].get('video_versions', []):
                                if video.get('url'):
                                    video_urls.append({
                                        'url': video.get('url'),
                                        'width': video.get('width') or 0,
                                        'height': video.get('height') or 0,
                                    })
                        elif media['type'] == 'image':
                            candidates = media['data'].get('image_versions2', {}).get('candidates', [])
                            for img in candidates:
                                if img.get('url'):
                                    image_urls.append({
                                        'url': img.get('url'),
                                        'width': img.get('width') or 0,
                                        'height': img.get('height') or 0,
                                    })
                    
                    if video_urls or image_urls:
                        break
                        
            except json.JSONDecodeError as e:
                if debug:
                    print(f"  [Script {idx}] JSON 解析失敗: {e}")
                continue
        
        # 下載媒體
        downloaded_files = []
        
        if video_urls:
            print(f"\n📹 找到 {len(video_urls)} 個影片版本")
            
            # 選擇最高品質
            def get_video_size(v):
                w = v.get('width') or 0
                h = v.get('height') or 0
                return w * h
            
            best_video = max(video_urls, key=get_video_size)
            
            print(f"📥 下載影片...")
            video_response = session.get(best_video['url'], headers=headers, stream=True)
            
            safe_title = re.sub(r'[<>:"/\\|?*]', '_', post_id)
            output_file = OUTPUT_DIR / f"threads_{safe_title}.mp4"
            
            with open(output_file, 'wb') as f:
                for chunk in video_response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            file_size = output_file.stat().st_size / (1024 * 1024)
            print(f"✅ 影片下載完成: {output_file.name} ({file_size:.2f} MB)")
            downloaded_files.append(str(output_file))
        
        elif image_urls:
            print(f"\n🖼️ 找到 {len(image_urls)} 個圖片版本")
            
            # 選擇最高品質
            def get_image_size(v):
                w = v.get('width') or 0
                h = v.get('height') or 0
                return w * h
            
            best_image = max(image_urls, key=get_image_size)
            
            print(f"📥 下載圖片...")
            image_response = session.get(best_image['url'], headers=headers, stream=True)
            
            safe_title = re.sub(r'[<>:"/\\|?*]', '_', post_id)
            output_file = OUTPUT_DIR / f"threads_{safe_title}.jpg"
            
            with open(output_file, 'wb') as f:
                for chunk in image_response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            file_size = output_file.stat().st_size / (1024 * 1024)
            print(f"✅ 圖片下載完成: {output_file.name} ({file_size:.2f} MB)")
            downloaded_files.append(str(output_file))
        
        else:
            # 嘗試直接從 HTML 提取
            print("\n嘗試直接從 HTML 提取媒體 URL...")
            
            video_match = re.search(r'"video_url"\s*:\s*"([^"]+)"', webpage)
            if video_match:
                video_url = video_match.group(1).replace('\\u0026', '&').replace('\\/', '/')
                print(f"  找到 video_url")
                
                video_response = session.get(video_url, headers=headers, stream=True)
                safe_title = re.sub(r'[<>:"/\\|?*]', '_', post_id)
                output_file = OUTPUT_DIR / f"threads_{safe_title}.mp4"
                
                with open(output_file, 'wb') as f:
                    for chunk in video_response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                file_size = output_file.stat().st_size / (1024 * 1024)
                print(f"✅ 影片下載完成: {output_file.name} ({file_size:.2f} MB)")
                downloaded_files.append(str(output_file))
            else:
                raise Exception("無法找到影片或圖片資料，可能需要登入或此貼文不包含媒體")
        
        elapsed_time = time.time() - start_time
        
        print(f"\n✅ 完成! 共下載 {len(downloaded_files)} 個檔案，耗時 {elapsed_time:.1f}s")
        
        test_results['threads_cookies'] = {
            'status': '✅ 成功',
            'time': f"{elapsed_time:.1f}s",
            'files': downloaded_files,
        }
        return True
        
    except Exception as e:
        elapsed_time = time.time() - start_time
        error_msg = str(e)[:100]
        print(f"\n❌ 解析失敗: {error_msg}")
        
        test_results['threads_cookies'] = {
            'status': f'❌ 失敗: {error_msg}',
        }
        
        import traceback
        traceback.print_exc()
        return False


def test_threads_with_cookies():
    """
    使用 Instagram/Threads cookies 測試私人貼文下載
    """
    cookies_path = Path('../cookies/threads.txt')
    
    # 也可以使用 Instagram cookies
    if not cookies_path.exists():
        cookies_path = Path('../cookies/instagram.txt')
    
    if not cookies_path.exists():
        print("⚠️ 找不到 cookies 檔案")
        print("   請在以下位置建立 cookies 檔案:")
        print(f"   - {Path('../cookies/threads.txt').absolute()}")
        print(f"   - {Path('../cookies/instagram.txt').absolute()}")
        print("\n   建立步驟:")
        print("   1. 安裝瀏覽器擴充套件 'Get cookies.txt LOCALLY'")
        print("   2. 登入 threads.com 或 instagram.com")
        print("   3. 點擊擴充套件匯出 cookies")
        print("   4. 將檔案存放於上述路徑")
        return False
    
    print(f"✅ 找到 cookies 檔案: {cookies_path}")
    
    # 使用手動解析方式 (因為 yt-dlp 官方尚未支援 Threads)
    test_url = TEST_URLS.get('threads_private', TEST_URLS['threads_public'])
    
    return parse_threads_with_cookies(test_url, cookies_path=str(cookies_path.absolute()))


# 執行測試
test_threads_with_cookies()